# Week 3-5

## 1. Write a program to implement Needleman-Wunsch for proteins
* You will need the blosum50 scoring matrix
* You can use any programming language
* Run this on HEAGAWGHEE versus PAWHEAE
* Compare this to page 23 in lecture 5
* Match the protein sequence SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL with PSPTMEAVTSVEASTASHPHSTSSYFATTYYHLY

In [1]:
import pandas as pd
Blosum50_df = pd.read_csv("blosum50.txt", delim_whitespace=True, header=None)
Blosum50_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,5,-2,-1,-2,-1,-1,-1,0,-2,-1,-2,-1,-1,-3,-1,1,0,-3,-2,0
1,-2,7,-1,-2,-4,1,0,-3,0,-4,-3,3,-2,-3,-3,-1,-1,-3,-1,-3
2,-1,-1,7,2,-2,0,0,0,1,-3,-4,0,-2,-4,-2,1,0,-4,-2,-3
3,-2,-2,2,8,-4,0,2,-1,-1,-4,-4,-1,-4,-5,-1,0,-1,-5,-3,-4
4,-1,-4,-2,-4,13,-3,-3,-3,-3,-2,-2,-3,-2,-2,-4,-1,-1,-5,-3,-1
5,-1,1,0,0,-3,7,2,-2,1,-3,-2,2,0,-4,-1,0,-1,-1,-1,-3
6,-1,0,0,2,-3,2,6,-3,0,-4,-3,1,-2,-3,-1,-1,-1,-3,-2,-3
7,0,-3,0,-1,-3,-2,-3,8,-2,-4,-4,-2,-3,-4,-2,0,-2,-3,-3,-4
8,-2,0,1,-1,-3,1,0,-2,10,-4,-3,0,-1,-1,-2,-1,-2,-3,2,-4
9,-1,-4,-3,-4,-2,-3,-4,-4,-4,5,2,-3,2,0,-3,-3,-1,-3,-1,4


In [2]:
def amino_acid_digitizing(seq):
    letter_digit_pair = []
    letter_digit_pair.extend([('A',0),('R',1),('N',2),('D',3),('C',4),('Q',5),\
                          ('E',6),('G',7),('H',8),('I',9),('L',10),('K',11),\
                          ('M',12),('F',13),('P',14),('S',15),('T',16),('W',17),('Y',18),('V',19)])
    letter_digit_df = pd.DataFrame(letter_digit_pair, columns=['letter', 'digit'])

    digits = []

    for letter in seq:
        digit = letter_digit_df['digit'].loc[letter_digit_df['letter'] == letter].values[0]
        digits.append(digit)

    return digits

In [3]:
def cost(a, b):
    return Blosum50_df[a][b]

In [63]:
def generateTable(seq1, seq2):
    digits1 = amino_acid_digitizing(seq1)
    digits2 = amino_acid_digitizing(seq2)
    
    table = [ [0 for i in range(len(seq1)+1)] for j in range(len(seq2)+1) ]
    for x in range(1,len(seq1)+1):
        table[0][x] = x*(-8)
    for y in range(1,len(seq2)+1):
        table[y][0] = y*(-8)
        
    for x in range(1,len(seq1)+1):
        for y in range(1,len(seq2)+1):
            f1 = table[y-1][x-1]+cost(digits1[x-1],digits2[y-1])
            f2 = table[y-1][x]-8
            f3 = table[y][x-1]-8
            table[y][x] = max(f1,f2,f3)
    return table

In [102]:
def backward(table, digits1, digits2):
    position = (len(table)-1,len(table[0])-1)
    path = [position]

    amino_digit1 = digits1[position[1]-1]
    amino_digit2 = digits2[position[0]-1]

    
    while (position[0] > 0 or position[1] > 0):
#         if position[0] == 0:
#             for x in range(position[1]-1,-1,-1):
#                 path = path + [(0,x)]
        if table[position[0]-1][position[1]-1] + cost(amino_digit1,amino_digit2) == table[position[0]][position[1]]:
            position = (position[0]-1,position[1]-1)
            path = path + [position]
        elif table[position[0]-1][position[1]] - 8 == table[position[0]][position[1]]:
                position = (position[0]-1,position[1])
                path = path + [position]
        elif table[position[0]][position[1]-1] - 8 == table[position[0]][position[1]]:
                position = (position[0],position[1]-1)
                path = path + [position]
        amino_digit1 = digits1[position[1]-1]
        amino_digit2 = digits2[position[0]-1]
    return path
            

In [111]:
backward(table,amino_acid_digitizing('MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY'),amino_acid_digitizing('TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI'))

## 2. Modify your program to implement the Smith-Waterman algorithm
* Again run this on HEAGAWGHEE versus PAWHEAE
* Compare this to page 5 in lecture 6
* Find the best local match between MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY and TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI

In [4]:
def generateTable_zero_limited(seq1, seq2):
    digits1 = amino_acid_digitizing(seq1)
    digits2 = amino_acid_digitizing(seq2)
    
    table = [ [0 for i in range(len(seq1)+1)] for j in range(len(seq2)+1) ]
    for x in range(1,len(seq1)+1):
        table[0][x] = 0
    for y in range(1,len(seq2)+1):
        table[y][0] = 0
        
    for x in range(1,len(seq1)+1):
        for y in range(1,len(seq2)+1):
            f1 = table[y-1][x-1]+cost(digits1[x-1],digits2[y-1])
            f2 = table[y-1][x]-8
            f3 = table[y][x-1]-8
            table[y][x] = max(f1,f2,f3) if max(f1,f2,f3) > 0 else 0
    return table

In [5]:
def backward_local(table, digits1, digits2):
    
    table_x_size = len(table)
    table_y_size = len(table[0])

    max_position = (0,0)
    max_value = -9999
    for x in range(table_x_size):
        for y in range(table_y_size):
            if table[x][y] >= max_value:
                max_value = table[x][y]
                max_position = (x,y)
    
    position = max_position
    path = [position]

    amino_digit1 = digits1[position[1]-1]
    amino_digit2 = digits2[position[0]-1]

    
    while (table[position[0]][position[1]] > 0):
#         if position[0] == 0:
#             for x in range(position[1]-1,-1,-1):
#                 path = path + [(0,x)]
        if table[position[0]-1][position[1]-1] + cost(amino_digit1,amino_digit2) == table[position[0]][position[1]]:
            position = (position[0]-1,position[1]-1)
            path = path + [position]
        elif table[position[0]-1][position[1]] - 8 == table[position[0]][position[1]]:
                position = (position[0]-1,position[1])
                path = path + [position]
        elif table[position[0]][position[1]-1] - 8 == table[position[0]][position[1]]:
                position = (position[0],position[1]-1)
                path = path + [position]
        amino_digit1 = digits1[position[1]-1]
        amino_digit2 = digits2[position[0]-1]
    return path
            

In [11]:
table1 = generateTable_zero_limited("MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY","TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI")
# table1

In [4]:
# backward_local(table1,amino_acid_digitizing('MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY'),amino_acid_digitizing('TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI'))

## 4. Program the following HMM to generate CG rich regions
<img src="hmm.png" width="600">

In [5]:
class HMM():
    def __init__(self, init_state):
        self.state = init_state
    
    def transform(self, trans_state):
        possibility = self.state.trans(trans_state)
        self.state = trans_state
        return possibility
    
    def emit(self, base):
        return self.state.emit(base)
     

class ATRich():
    
    def __init__(self):
        self.base_dic = {
            "A" : {
                "possibility" : 0.2698
            },
            "T" : {
                "possibility" : 0.3237
            },
            "C" : {
                "possibility" : 0.2080
            },
            "G" : {
                "possibility" : 0.1985
              }  
        }
        
        self.cg_trans = 0.0002
        
    def trans(self, state):
        if type(state) == type(CGRich()):
            return self.cg_trans
        else:
            raise('wrong type of state')
    
    def emit(self, base):
        return self.base_dic.get(base).get('possibility')

class CGRich():
    def __init__(self):
        self.base_dic = {
            "A" : {
                "possibility" : 0.2459
            },
            "T" : {
                "possibility" : 0.2079
            },
            "C" : {
                "possibility" : 0.2478
            },
            "G" : {
                "possibility" : 0.2984
              }  
        }
        
        self.at_trans = 0.0003
        
    def trans(self, state):
        if type(state) == type(ATRich()):
            return self.at_trans
        else:
            raise('wrong type of state')
    
    def emit(self, base):
        return self.base_dic.get(base).get('possibility')
    

In [6]:
hmm = HMM(CGRich())
hmm.transform(ATRich())
hmm.emit('A')
hmm.transform(CGRich())
hmm.emit('A')

0.2459

In [20]:
# file = open('phaseLambda.fasta.txt', 'r')
with open ('phaseLambda.fasta.txt', 'r') as file:
    seq = file.read().replace('\n', '')


In [57]:
import math

def viterbi_forward(seq):
    table = [(-math.log(0.5), -math.log(0.5))]
    index = 0
    cg = CGRich()
    at = ATRich()
    for base in seq:
        s1v = table[index][0]
        s2v = table[index][1]
        s1p = min(s1v-math.log(cg.emit(base)), s2v-math.log(at.cg_trans)-math.log(cg.emit(base)))
        s2p = min(s2v-math.log(at.emit(base)), s1v-math.log(cg.at_trans)-math.log(at.emit(base)))
#         print(s1v-math.log(cg.emit(base)), s2v-math.log(at.cg_trans)-math.log(cg.emit(base)))
#         if(s1v-math.log(cg.emit(base)) > s2v-math.log(at.cg_trans)-math.log(cg.emit(base))):
#             print('true')
#         if(s2v-math.log(at.emit(base)) > s2v-math.log(at.cg_trans)-math.log(cg.emit(base))):
#             print('true')
        s1p = round(s1p,3)
        s2p = round(s2p,3)
        table.append((s1p, s2p))
        index += 1
#         print(index)
    return table


In [58]:
posibility_table = viterbi_forward(seq)

In [155]:
def viterbi_backward(seq, table):
    
    cg = CGRich()
    at = ATRich()
    
    states = []
    
    current_min = min(table[-1])
    if current_min == table[-1][0]:
        states.append(0)
    else:
        states.append(1)
    index = 0
    for x in range(len(seq)-1, 0, -1):
        base = seq[x]
        paths = table[x]
        
        x1 = round(current_min+math.log(cg.emit(base)),3)
        x2 = round(current_min+math.log(at.cg_trans)+math.log(cg.emit(base)),3)
        x3 = round(current_min+math.log(at.emit(base)),3)
        x4 = round(current_min+math.log(cg.at_trans)+math.log(at.emit(base)),3)
        
#         if index > 0 and index < 200:
#             print(current_min)
#             print(x1, x2, x3, x4)
#             print(paths)
#             print()
        index += 1
        
        if x1 == paths[0] or x4 == paths[0]:
            states.append(0)
            current_min = paths[0]
            
        elif x3 == paths[1] or x2 == paths[1]:
            states.append(1)
            current_min = paths[1]
                
    return states
        
        

In [156]:
states = viterbi_backward(seq, posibility_table)

In [157]:
len(posibility_table)
len(seq)

48502

In [159]:
len(states)

48502

In [160]:
import csv

state_rows = []
index = 0
row = []
for state in states:
    row.append(state)
    if index == 69:
        state_rows.append(row)
        row = []  
    index += 1
    if index == 70:
        index = 0
    
    
# (state_rows)
with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(state_rows)

In [142]:
# state_rows